In [1]:
import json

# preps in which 1st object has greater z value than 2nd
deeper_preps = [' behind ', ' inside ']

# 1st has lower z value
shallower_preps = [' outside ', ' front ', ' in front of ', ' against ']

In [2]:
with open('subset_data\\region_graphs.json') as region_graph, open('subset_data\objects.json') as objects:
    data = json.load(region_graph)
    mini = json.load(objects)


In [4]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image as PIL_Image
import requests
from StringIO import StringIO
from visual_genome import models, api, local, utils

SyntaxError: invalid syntax (pyplot.py, line 2387)

In [ ]:
annotated_regions = []

In [ ]:
def get_image_data(id):
    """
    Get data about an image.
    """
    
    data = utils.retrieve_data('/api/v0/images/' + str(id))
    
    if 'detail' in data and data['detail'] == 'Not found.':
        return None
    image = utils.parse_image_data(data)

    return image

# print(get_image_data(10))

In [ ]:
def annotate(region):
#     print(region.keys())
#     annotated_regions.append(region)
#     image = get_image_data(region['image_id'])
#     response = requests.get(image.url)
#     img = PIL_Image.open(StringIO(response.content))
#     plt.imshow(img)
#     ax = plt.gca()
    print(region['image_id'])
# print(annotated_regions)

def get_object_details(name, details, idx):
    obj_to_add = [name,\
                      details[0],\
                      details[1],\
                      idx,\
                      details[2],\
                      details[3]
                     ]
    
    return obj_to_add

In [ ]:
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
def visualize_regions(image, regions):
    response = requests.get(image.url)
    img = PIL_Image.open(StringIO(response.content))
    plt.imshow(img)
    ax = plt.gca()
    for region in regions:
        ax.add_patch(Rectangle((region.x, region.y),
                               region.width,
                               region.height,
                               fill=False,
                               edgecolor='red',
                               linewidth=3))
        ax.text(region.x, region.y, region.phrase, style='italic', bbox={'facecolor':'white', 'alpha':0.7, 'pad':10})
    fig = plt.gcf()
    plt.tick_params(labelbottom='off', labelleft='off')
    plt.show()
visualize_regions(image, regions[:8])

In [ ]:
to_annotate = {}

for image, objs in zip(data, mini):
    wannabe_objetcs = {}
    for o in objs['objects']:
         wannabe_objetcs[o['names'][0]]=  (o['x'], o['y'], o['h'], o['w'])
    regions = image['regions']
    annotate_reg = []
    annotate_shallow = []
    annotate_deep = []
    for region in regions:
        region_added = False
        for word in deeper_preps:
            if word in region['phrase']:
#                 print(region['phrase'])

                temp = sorted(region['synsets'], key=lambda k: k['entity_idx_start'], reverse=True)
                for idx, ele in enumerate(temp):
                    low_life = ele['entity_name'].lower()
                    if low_life in wannabe_objetcs.keys():
                        if not region_added:
                            annotate_reg.append(region)
                            region_added = True
                        obj_to_add = get_object_details(ele['entity_name'], wannabe_objetcs[low_life], idx)
                        annotate_deep.append(obj_to_add)
                        
                        if idx == 0 :
                            annotate_shallow.append(obj_to_add)
                        else :
                            annotate_deep.append(obj_to_add)
                        
#                         print(region['phrase'])
                        
#                         reg.appen(region)
#                         annotate_obj.appen()
#                         try:
#                             to_annotate[region['image_id']].append(region)
                        
#                             annotate(region)
#                         print(region['phrase'])
                        
                        super_string = ''
                        super_string += ele['entity_name'] + ';'
                        super_string += ' x = ' + str(wannabe_objetcs[low_life][0])
                        super_string += ', y = ' + str(wannabe_objetcs[low_life][1])
                        super_string += ', z = ' + str(idx)
#                         print(super_string)
#                 print('----xxx----')


        for word in shallower_preps:
            if word in region['phrase']:
#                 print(region['phrase'])

                temp = sorted(region['synsets'], key=lambda k: k['entity_idx_start'])
                for idx, ele in enumerate(temp):
                    low_life = ele['entity_name'].lower()
                    if low_life in wannabe_objetcs:
                        obj_to_add = get_object_details(ele['entity_name'], wannabe_objetcs[low_life],idx)
                        if idx == 0 :
                            annotate_shallow.append(obj_to_add)
                        else :
                            annotate_deep.append(obj_to_add)
                        super_string = ''
                        super_string += ele['entity_name'] + ';'
                        super_string += ' x = ' + str(wannabe_objetcs[low_life][0])
                        super_string += ', y = ' + str(wannabe_objetcs[low_life][1])
                        super_string += ', z = ' + str(idx)
#                         print(super_string)
                print(annotate_shallow)
                
                print("-------")
                print(annotate_deep)
                print("-----xxxxx----")

#                 print('----xxx----')
#     print(annotate_reg, annotate_deep, annotate_shallow)